In [1]:
#Importing Necessary Packages

import numpy as np
import pandas as pd
import sys
import gc
import pickle
import datetime
from itertools import product
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
sales_train_merged = pd.read_pickle('sales_train_merged.pkl')
sales_train = pd.read_csv('data/sales_train_v2.csv')

In [3]:
sales_train_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12074537 entries, 0 to 12074536
Data columns (total 8 columns):
date_block_num      int8
shop_id             int8
item_id             int16
item_cnt_month      float16
city_id             int8
item_category_id    int8
type_id             int8
subtype_id          int8
dtypes: float16(1), int16(1), int8(6)
memory usage: 207.3 MB


In [4]:
# Function to create lag features
def create_lags(df, lags, column):
    temp_df = df[['date_block_num','shop_id','item_id',column]]
    cnt = 1
    for lag in lags:
        pushed_df = temp_df.copy()
        pushed_df.columns = ['date_block_num','shop_id','item_id', column+'_lag_'+str(cnt)]
        pushed_df['date_block_num'] += lag
        df = pd.merge(df, pushed_df, on=['date_block_num','shop_id','item_id'], how='left')
        cnt+=1
    return df

In [5]:
#Creating Lags

In [6]:
months_3 = [1,2]

In [7]:
sales_train_merged = create_lags(sales_train_merged, months_3, 'item_cnt_month')

In [8]:
sales_train_merged.head()

,date_block_num,shop_id,item_id,item_cnt_month,city_id,item_category_id,type_id,subtype_id,item_cnt_month_lag_1,item_cnt_month_lag_2
0,0,2,19,0.0,0,40,11,4,NaN,NaN
1,0,2,27,1.0,0,19,5,10,NaN,NaN
2,0,2,28,0.0,0,30,8,55,NaN,NaN
3,0,2,29,0.0,0,23,5,16,NaN,NaN
4,0,2,32,0.0,0,40,11,4,NaN,NaN


In [9]:
grouped_df = sales_train_merged.groupby(['date_block_num']).agg({'item_cnt_month': ['mean']})
grouped_df.columns = [ 'date_item_cnt_month_mean' ]
grouped_df.reset_index(inplace=True)
sales_train_merged = pd.merge(sales_train_merged, grouped_df, on=['date_block_num'], how='left')
sales_train_merged['date_item_cnt_month_mean'] = sales_train_merged['date_item_cnt_month_mean'].astype(np.float16)
sales_train_merged = create_lags(sales_train_merged, [months_3[0]], 'date_item_cnt_month_mean')
sales_train_merged.drop(['date_item_cnt_month_mean'], axis=1, inplace=True)

In [ ]:
grouped_df = sales_train_merged.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
grouped_df.columns = [ 'date_item_item_cnt_month_mean' ]
grouped_df.reset_index(inplace=True)
sales_train_merged = pd.merge(sales_train_merged, grouped_df, on=['date_block_num','item_id'], how='left')
sales_train_merged['date_item_item_cnt_month_mean'] = sales_train_merged['date_item_item_cnt_month_mean'].astype(np.float16)
sales_train_merged = create_lags(sales_train_merged, months_3, 'date_item_item_cnt_month_mean')
sales_train_merged.drop(['date_item_item_cnt_month_mean'], axis=1, inplace=True)

In [21]:
grouped_df = sales_train_merged.groupby(['date_block_num', 'shop_id']).agg({'item_cnt_month': ['mean']})
grouped_df.columns = [ 'date_shop_item_cnt_month_mean' ]
grouped_df.reset_index(inplace=True)
sales_train_merged = pd.merge(sales_train_merged, grouped_df, on=['date_block_num','shop_id'], how='left')
sales_train_merged['date_shop_item_cnt_month_mean'] = sales_train_merged['date_shop_item_cnt_month_mean'].astype(np.float16)
sales_train_merged = create_lags(sales_train_merged, month_3, 'date_shop_item_cnt_month_mean')
sales_train_merged.drop(['date_shop_item_cnt_month_mean'], axis=1, inplace=True)

In [ ]:
grouped_df = sales_train_merged.groupby(['date_block_num', 'item_category_id']).agg({'item_cnt_month': ['mean']})
grouped_df.columns = [ 'date_item_cat_item_cnt_month_mean' ]
grouped_df.reset_index(inplace=True)
sales_train_merged = pd.merge(sales_train_merged, grouped_df, on=['date_block_num','item_category_id'], how='left')
sales_train_merged['date_item_cat_item_cnt_month_mean'] = sales_train_merged['date_item_cat_item_cnt_month_mean'].astype(np.float16)
sales_train_merged = create_lags(sales_train_merged, [months_3[0]], 'date_item_cat_item_cnt_month_mean')
sales_train_merged.drop(['date_item_cat_item_cnt_month_mean'], axis=1, inplace=True)

In [ ]:
grouped_df = sales_train_merged.groupby(['date_block_num', 'shop_id', 'item_category_id']).agg({'item_cnt_month': ['mean']})
grouped_df.columns = ['date_shop_item_cat_item_cnt_mean']
grouped_df.reset_index(inplace=True)
sales_train_merged = pd.merge(sales_train_merged, grouped_df, on=['date_block_num', 'shop_id', 'item_category_id'], how='left')
sales_train_merged['date_shop_item_cat_item_cnt_mean'] = sales_train_merged['date_shop_item_cat_item_cnt_mean'].astype(np.float16)
sales_train_merged = create_lags(sales_train_merged, [months_3[0]], 'date_shop_item_cat_item_cnt_mean')
sales_train_merged.drop(['date_shop_item_cat_item_cnt_mean'], axis=1, inplace=True)

In [ ]:
grouped_df = sales_train_merged.groupby(['date_block_num', 'shop_id', 'type_code']).agg({'item_cnt_month': ['mean']})
grouped_df.columns = ['date_shop_type_item_cnt_meam']
grouped_df.reset_index(inplace=True)
sales_train_merged = pd.merge(sales_train_merged, grouped_df, on=['date_block_num', 'shop_id', 'type_code'], how='left')
sales_train_merged['date_shop_type_item_cnt_meam'] = sales_train_merged['date_shop_type_item_cnt_meam'].astype(np.float16)
sales_train_merged = create_lags(sales_train_merged, [months_3[0]], 'date_shop_type_item_cnt_meam')
sales_train_merged.drop(['date_shop_type_item_cnt_meam'], axis=1, inplace=True)

In [ ]:
grouped_df = sales_train_merged.groupby(['date_block_num', 'shop_id', 'subtype_code']).agg({'item_cnt_month': ['mean']})
grouped_df.columns = ['date_shop_subtype_item_cnt_mean']
grouped_df.reset_index(inplace=True)
sales_train_merged = pd.merge(sales_train_merged, grouped_df, on=['date_block_num', 'shop_id', 'subtype_code'], how='left')
sales_train_merged['date_shop_subtype_item_cnt_mean'] = sales_train_merged['date_shop_subtype_item_cnt_mean'].astype(np.float16)
sales_train_merged = create_lags(sales_train_merged, [months_3[0]], 'date_shop_subtype_item_cnt_mean')
sales_train_merged.drop(['date_shop_subtype_item_cnt_mean'], axis=1, inplace=True)

In [ ]:
grouped_df = sales_train_merged.groupby(['date_block_num', 'city_code']).agg({'item_cnt_month': ['mean']})
grouped_df.columns = [ 'date_city_item_cnt_month_mean' ]
grouped_df.reset_index(inplace=True)
sales_train_merged = pd.merge(sales_train_merged, grouped_df, on=['date_block_num', 'city_code'], how='left')
sales_train_merged['date_city_item_cnt_month_mean'] = sales_train_merged['date_city_item_cnt_month_mean'].astype(np.float16)
sales_train_merged = create_lags(sales_train_merged, [months_3[0]], 'date_city_item_cnt_month_mean')
sales_train_merged.drop(['date_city_item_cnt_month_mean'], axis=1, inplace=True)

In [ ]:
grouped_df = sales_train_merged.groupby(['date_block_num', 'item_id', 'city_code']).agg({'item_cnt_month': ['mean']})
grouped_df.columns = [ 'date_item_city_item_cnt_mean' ]
grouped_df.reset_index(inplace=True)
sales_train_merged = pd.merge(sales_train_merged, grouped_df, on=['date_block_num', 'item_id', 'city_code'], how='left')
sales_train_merged['date_item_city_item_cnt_mean'] = sales_train_merged['date_item_city_item_cnt_mean'].astype(np.float16)
sales_train_merged = create_lags(sales_train_merged, [1], 'date_item_city_item_cnt_mean')
sales_train_merged.drop(['date_item_city_item_cnt_mean'], axis=1, inplace=True)

In [ ]:
grouped_df = sales_train_merged.groupby(['date_block_num', 'type_code']).agg({'item_cnt_month': ['mean']})
grouped_df.columns = [ 'date_type_item_cnt_month_mean' ]
grouped_df.reset_index(inplace=True)
sales_train_merged = pd.merge(sales_train_merged, grouped_df, on=['date_block_num', 'type_code'], how='left')
sales_train_merged['date_type_item_cnt_month_mean'] = sales_train_merged['date_type_item_cnt_month_mean'].astype(np.float16)
sales_train_merged = create_lags(sales_train_merged, [1], 'date_type_item_cnt_month_mean')
sales_train_merged.drop(['date_type_item_cnt_month_mean'], axis=1, inplace=True)

In [ ]:
grouped_df = sales_train_merged.groupby(['date_block_num', 'subtype_code']).agg({'item_cnt_month': ['mean']})
grouped_df.columns = [ 'date_subtype_item_cnt_month_mean' ]
grouped_df.reset_index(inplace=True)
sales_train_merged = pd.merge(sales_train_merged, grouped_df, on=['date_block_num', 'subtype_code'], how='left')
sales_train_merged['date_subtype_item_cnt_month_mean'] = sales_train_merged['date_subtype_item_cnt_month_mean'].astype(np.float16)
sales_train_merged = create_lags(sales_train_merged, [1], 'date_subtype_item_cnt_month_mean')
sales_train_merged.drop(['date_subtype_item_cnt_month_mean'], axis=1, inplace=True)

In [ ]:
sales_train_merged.head()

_______________________________

In [ ]:
#More Feature Engineering

In [ ]:
grouped_df = train.groupby(['item_id']).agg({'item_price': ['mean']})
grouped_df.columns = ['item_item_price_mean']
grouped_df.reset_index(inplace=True)
sales_train_merged = pd.merge(sales_train_merged, grouped_df, on=['item_id'], how='left')
sales_train_merged['item_item_price_mean'] = sales_train_merged['item_item_price_mean'].astype(np.float16)

In [ ]:
grouped_df = train.groupby(['date_block_num','item_id']).agg({'item_price': ['mean']})
grouped_df.columns = ['date_item_item_price_mean']
grouped_df.reset_index(inplace=True)
sales_train_merged = pd.merge(sales_train_merged, grouped_df, on=['date_block_num','item_id'], how='left')
sales_train_merged['date_item_item_price_mean'] = sales_train_merged['date_item_item_price_mean'].astype(np.float16)

In [ ]:
lags = months_3
sales_train_merged = create_lags(sales_train_merged, lags, 'date_item_item_price_mean')

In [ ]:
for i in lags:
    sales_train_merged['price_change_lag_'+str(i)] = \
        (sales_train_merged['date_item_avg_item_price_lag_'+str(i)] - sales_train_merged['item_avg_item_price']) / sales_train_merged['item_avg_item_price']

def pick_feature(row):
    for i in lags:
        if row['price_change_lag_'+str(i)]:
            return row['price_change_lag_'+str(i)]
    return 0

In [ ]:
sales_train_merged['price_change_lag'] = sales_train_merged.apply(pick_feature, axis=1)
sales_train_merged['price_change_lag'] = sales_train_merged['price_change_lag'].astype(np.float16)
sales_train_merged['price_change_lag'].fillna(0, inplace=True)

drop_list = ['item_avg_item_price', 'date_item_item_price_mean']
for i in lags:
    drop_list += ['date_item_avg_item_price_lag_'+str(i)]
    drop_list += ['price_change_lag_'+str(i)]

sales_train_merged.drop(drop_list, axis=1, inplace=True)

In [ ]:
grouped_df = sales_train.groupby(['date_block_num','shop_id']).agg({'revenue': ['sum']})
grouped_df.columns = ['date_shop_total_revenue']
grouped_df.reset_index(inplace=True)
sales_train_merged = pd.merge(sales_train_merged, grouped_df, on=['date_block_num','shop_id'], how='left')
sales_train_merged['date_shop_total_revenue'] = sales_train_merged['date_shop_total_revenue'].astype(np.float32)

In [ ]:
grouped_df = grouped_df.groupby(['shop_id']).agg({'date_shop_total_revenue': ['mean']})
grouped_df.columns = ['shop_revenue_mean']
grouped_df.reset_index(inplace=True)
sales_train_merged = pd.merge(sales_train_merged, grouped_df, on=['shop_id'], how='left')
sales_train_merged['shop_revenue_mean'] = sales_train_merged['shop_revenue_mean'].astype(np.float32)

In [ ]:
sales_train_merged['revenue_change'] = (sales_train_merged['date_shop_total_revenue'] - sales_train_merged['shop_revenue_mean']) / sales_train_merged['shop_revenue_mean']
sales_train_merged['revenue_change'] = sales_train_merged['revenue_change'].astype(np.float16)
sales_train_merged = create_lags(sales_train_merged, [1], 'revenue_change')
sales_train_merged.drop(['date_shop_total_revenue','shop_revenue_mean','revenue_change'], axis=1, inplace=True)

In [ ]:
sales_train_merged['month'] = sales_train_merged['date_block_num'] % 12
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
sales_train_merged['days'] = sales_train_merged['month'].map(days).astype(np.int8)

In [ ]:
temp_dict = {}
sales_train_merged['item_shop_prev_sale'] = -1
sales_train_merged['item_shop_prev_sale'] = sales_train_merged['item_shop_prev_sale'].astype(np.int8)
for row_id, row in sales_train_merged.iterrows():    
    key = str(row.item_id)+' '+str(row.shop_id)
    if key not in temp_dict:
        if row.item_cnt_month!=0:
            temp_dict[key] = row.date_block_num
    else:
        last_date_block_num = temp_dict[key]
        sales_train_merged.at[row_id, 'item_shop_prev_sale'] = row.date_block_num - last_date_block_num
        temp_dict[key] = row.date_block_num

In [ ]:
temp_dict = {}
sales_train_merged['prev_item_sale'] = -1
sales_train_merged['prev_item_sale'] = sales_train_merged['prev_item_sale'].astype(np.int8)
for row_id, row in sales_train_merged.iterrows():    
    key = row.item_id
    if key not in temp_dict:
        if row.item_cnt_month!=0:
            temp_dict[key] = row.date_block_num
    else:
        last_date_block_num = temp_dict[key]
        if row.date_block_num>last_date_block_num:
            sales_train_merged.at[row_id, 'prev_item_sale'] = row.date_block_num - last_date_block_num
            temp_dict[key] = row.date_block_num

In [ ]:
sales_train_merged['item_shop_first_sale'] = sales_train_merged['date_block_num'] - sales_train_merged.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
sales_train_merged['item_first_sale'] = sales_train_merged['date_block_num'] - sales_train_merged.groupby('item_id')['date_block_num'].transform('min')

In [ ]:
sales_train_merged = sales_train_merged[sales_train_merged.date_block_num > 11]

def fill_na(df):
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            if ('item_cnt' in col):
                df[col].fillna(0, inplace=True)         
    return df

sales_train_merged = fill_na(sales_train_merged)

In [ ]:
sales_train_merged.to_pickle('sales_train_final.pkl')